## 8. Violin plot
Code for creating the violin plots to visualize the number of interventions in conversations and conversation lengths of airlines in order to compare American Airlines to its competitors  
Also we chose the competitors of American Airlines by the most mentioned Airlines

Two tables are added to the database to help make the plots:  
Names: contains the full name of the airline and the corresponding airline ID  
ConvoInfo: contains per each conversation the ID of the airline, the length of the conversation and the number of interventions by the airline in that conversation  

**input**: conversation pickle file and database  
**output**: violin plots

In [ ]:
import numpy as np
import pandas as pd
import pickle
import sqlite3

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after importing seaborn

In [ ]:
# connecting to the database and 
# accessing the pickle file with the conversation data

conn = sqlite3.connect('core.db')
all_convos = pickle.load(open("obj/conversations_with_scores.p", "rb"))

## First part: Competitors
This part of the code was used to choose the competitors for American Airlines.

In [ ]:
# a function to get the number of conversations for each airline

def GetAirlineConversations(convos, airlines):
    all_convos = []
    for airline in airlines:
        airline_convos = []
        for convo in convos:
            found = False
            for info in convo:
                for user_id in info:
                    if info[1] == airline and not found:
                        airline_convos.append(convo)
                        found = True
        all_convos.append(len(airline_convos))
    return all_convos

In [ ]:
# queries to get the number of mentions for each airline

AA = pd.read_sql_query('''select count(id) from twitter where text like "%American%Air%"''', conn) ["count(id)"][0]
KLM = pd.read_sql_query('''select count(id) from twitter where text like "%KLM%"''', conn) ["count(id)"][0]
France = pd.read_sql_query('''select count(id) from twitter where text like "%Air%France%"''', conn) ["count(id)"][0]
British = pd.read_sql_query('''select count(id) from twitter where text like "%British%Airways%"''', conn) ["count(id)"][0]
Lufthansa = pd.read_sql_query('''select count(id) from twitter where text like "%Lufthansa%"''', conn) ["count(id)"][0]
Airberlin = pd.read_sql_query('''select count(id) from twitter where text like "%AirBerlin%"''', conn) ["count(id)"][0]
AirB_assist = pd.read_sql_query('''select count(id) from twitter where text like "%AirBerlin%assist%"''', conn) ["count(id)"][0]
easyJet = pd.read_sql_query('''select count(id) from twitter where text like "%easy%Jet%"''', conn) ["count(id)"][0]
RyanAir = pd.read_sql_query('''select count(id) from twitter where text like "%Ryan%Air%"''', conn) ["count(id)"][0]
SingaporeAir = pd.read_sql_query('''select count(id) from twitter where text like "%Singapore%Air%"''', conn) ["count(id)"][0]
Qantas = pd.read_sql_query('''select count(id) from twitter where text like "%Qantas%"''', conn) ["count(id)"][0]
EtihadAirways = pd.read_sql_query('''select count(id) from twitter where text like "%Etihad%Airways%"''', conn) ["count(id)"][0]
VirginAtlantic = pd.read_sql_query('''select count(id) from twitter where text like "%Virgin%Atlantic%"''', conn) ["count(id)"][0]


In [ ]:
airlines = [56377143, 106062176, 18332190, 22536055, 124476322, 26223583, 2182373406,
            38676903, 1542862735, 253340062, 218730857, 45621423, 20626359]
airline_names = ["KLM", "Air France", "British Airways", "American Airlines", "Lufthansa", 
                 "Air Berlin", "Air Berlin assist", "easyJet", "Ryanair", "Singapore Airlines", 
                 "Qantas", "Etihad Airways", "Virgin Atlantic"]
airline_convos = GetAirlineConversations(all_convos, airlines)
airline_mentions = [KLM, France, British, AA, Lufthansa, Airberlin, AirB_assist, 
                    easyJet, RyanAir, SingaporeAir, Qantas, EtihadAirways, VirginAtlantic]

In [ ]:
# creating a dataframe with airline names, number of mentions, 
# number of conversations and a reply ratio to get a quick overview

# we chose the competitors based on the values in this table when all the data was used

con_men = pd.DataFrame()
con_men['Airline'] = pd.Series(airline_names).values
con_men['Number of mentions'] = pd.Series(airline_mentions).values
con_men['Number of conversations'] = pd.Series(airline_convos).values
con_men['Ratio'] = con_men['Number of conversations']/con_men['Number of mentions']
con_men.sort_values(['Number of mentions'], ascending=False)

## Second part: Violin plots
This part of the code was used to create the violin plots for the lengths of conversations of airlines and number of interventions within a conversation by airlines.

In [ ]:
# creating a dictionary with airline names and corresponding ids and 
# then using it to create the dataframe with the same information

name_dict = dict()
for i in range(len(airline_names)):
    name_dict[airline_names[i]] = airlines[i]

Name_decoded = pd.DataFrame(list(name_dict.items()), columns=['Name', 'ID'])
Name_decoded

In [ ]:
# saving the dataframe as a table into the database

Name_decoded.to_sql("AirlineInfo", conn, if_exists= "replace")

In [ ]:
# a function to get for each conversation the airline that was involved and the length of the conversation

def GetAirlineConversationsLengths(convos, airlines):
    airline_convos = []
    for airline in airlines:
        for convo in convos:
            found = False
            for tweet in convo:
                if tweet[1] == airline and not found:
                    airline_convos.append((airline, len(convo)))
                    found = True
    return airline_convos

# a function to get for each conversation the airline that was involved and how many times the airline intervened

def GetNumberOfInterventions(convos, airlines):
    num_interventions = []
    for airline in airlines:
        for convo in convos:
            intervention = 0
            for tweet in convo:
                if tweet[1] == airline:
                    intervention += 1
            if intervention != 0:
                num_interventions.append((airline, intervention))
    return num_interventions


In [ ]:
# creating a dataframe with the airline and length of conversation for each conversation
convo_lengths = GetAirlineConversationsLengths(all_convos, airlines)
df_lengths = pd.DataFrame(convo_lengths, columns=["airline", "length"])

# creating a dataframe with the airline and number of intervention for each conversation
interventions = GetNumberOfInterventions(all_convos, airlines)
df_inter = pd.DataFrame(interventions, columns = ['airline', 'num_inter'])

# combining the dataframes so that one dataframe has information about lenghts and interventions
df_lengths["num_inter"] = df_inter["num_inter"]
df_lengths.head()

In [ ]:
# saving the combined dataframe as a table into the database

df_lengths.to_sql("ConvoInfo", conn, if_exists= "replace")

In [ ]:
# query to have a dataframe with airline names, conversation lengths and number of interventions

query = '''
SELECT a.name AS airline, c.length AS length, c.num_inter AS num_inter
FROM ConvoInfo AS c, AirlineInfo AS a
WHERE c.airline = a.id;
'''
info = pd.read_sql_query(query, conn)

In [ ]:
# function to reject the outliers in the data

def reject_outliers(data, m=8):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [ ]:
# a list with the defined competitors based on the result including all data from the table in the first part and American Airlines

competitors = ['KLM', 'British Airways', 'American Airlines', 'Lufthansa', 
               'easyJet', 'Ryanair', 'Qantas']

In [ ]:
# rejecting the outliers in the length column
indices = reject_outliers(info["length"]).index

# only keeping the non-outliers in the table
combined = info.loc[info.index.isin(indices)]

# only keeping the conversations that include AA or the defined competitors
final = combined.loc[combined['airline'].isin(competitors)]

In [ ]:
# setting up a palette for the violinplot

my_palette = {airline: '#d3494e' if airline == "American Airlines" else "#658cbb" for airline in final.airline.unique()}

In [ ]:
# setting up an order in the plots based on how many conversations each airline had

count_order = final.groupby("airline")["length"].count().sort_values(ascending=False).index

In [ ]:
sns.set(font_scale = 1.8)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = 20, 13
ax = sns.violinplot(data=final, x=final["airline"], y=final["length"],  
               scale="width", palette=my_palette, order=count_order)
ax.set_title("Violin Plot for the conversation lengths of American Airlines and their competitors", 
             weight = "bold", size = 24);
ax.set_xlabel("Airline", size = 20)
ax.set_ylabel("Conversation length", size = 20);
ax.set(xticklabels=['American Airlines', 'British Airways', 'Ryanair', 'easyJet', 'KLM', 'Qantas', 'Lufthansa']);
# plt.savefig('airline_convos_new_new.pdf', bbox_inches = "tight")

In [ ]:
sns.set(font_scale = 1.8)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = 20, 13
ax = sns.violinplot(data=final, x=final["airline"], y=final["num_inter"], scale="width", palette=my_palette,  
                    inner="box", order=count_order)
ax.set_title("Violin Plot for the number of interventions in a conversation for American Airlines and their competitors", 
             weight = "bold", size = 18);
ax.set_xlabel("Airline", size = 20)
ax.set_ylabel("Number of interventions", size = 20);
# plt.savefig('airline_inters.pdf', bbox_inches = "tight")

In [ ]:
print('Done')